## Recipe Chatbot

###Secrets and imports

In [50]:
!ls # check location is right

data  data_cleaning.py	model.py  recipe_chatbot.ipynb	recipe_index.faiss


In [53]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    data/context_texts.json
	deleted:    data/formatted_recipes.json
	deleted:    data/recipe_ids.json
	modified:   recipe_chatbot.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/food.com/

no changes added to commit (use "git add" and/or "git commit -a")


In [51]:
from google.colab import userdata

git_token = userdata.get('git-token')
hf_token = userdata.get('hf-token')

In [ ]:
!pip install -q sentence-transformers faiss-cpu transformers huggingface_hub

from huggingface_hub import login
login(token=hf_token)

import json
import numpy as np
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.8 MB/s eta 0:00:00


In [ ]:
if not torch.cuda.is_available():
  print("Runtime not set to GPU")
else:
  print("good")

good


### Embeddings

In [ ]:
# choosing embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# fns

# given a recipe, get an embedding of it for search
def get_recipe_emb(recipe, model):
    title = recipe["title"]
    category = recipe.get("category", "")
    tags = ", ".join(recipe["tags"]) if recipe["tags"][0] else ""
    desc = recipe.get("description", "")
    ingredients = ", ".join(recipe["ingredients"])
    search_text = f"{title} - {category} - {tags} - {desc} - {ingredients}"
    return model.encode(search_text)

# given a recipe, get the context that will be returned if chosen by RAG
def get_recipe_context(recipe):
    title = recipe["title"]
    tags = ", ".join(recipe["tags"]) if recipe["tags"][0] else ""
    desc = recipe.get("description", "")
    ingredients = ", ".join(recipe["ingredients"])
    steps = ", ".join(recipe["steps"])
    return f"Title: {title}\nDescription: {desc}\nTags: {tags}\nIngredients: {ingredients}\nSteps: {steps}"

# Given a model and dataset of recipes, save the recipes' IDs,
# context texts, and indexed search vectors (embeddings)
def save_embeddings(model, recipes_path="/content/formatted_recipes.json"):
    with open(recipes_path) as f:
        recipes = json.load(f)
    recipe_ids, search_vecs, context_texts = [], [], []
    for idx, recipe in enumerate(recipes):
        recipe_ids.append(idx)
        search_vecs.append(get_recipe_emb(recipe, model))
        context_texts.append(get_recipe_context(recipe))
    emb_matrix = np.array(search_vecs).astype("float32")
    index = faiss.IndexFlatL2(emb_matrix.shape[1])
    index.add(emb_matrix)
    faiss.write_index(index, "/content/recipe_index.faiss")
    with open("/content/context_texts.json", "w") as f:
        json.dump(context_texts, f)
    with open("/content/recipe_ids.json", "w") as f:
        json.dump(recipe_ids, f)

# Get top recipes in terms of similarity to user's query
def get_top_recipes(model, user_query, n_recipes=3):
    encoded_query = model.encode([user_query])
    index = faiss.read_index("/content/recipe_index.faiss")
    D, I = index.search(np.array(encoded_query, dtype="float32"), k=n_recipes)
    with open("/content/context_texts.json") as f:
        context_texts = json.load(f)
    with open("/content/formatted_recipes.json") as f:
        recipes = json.load(f)
    return [context_texts[i] for i in I[0]]

# Create a prompt using the user query and the top recipe context(s)
def create_prompt(user_query, top_contexts):
    return f"""<s>[INST]
You are a friendly cooking assistant that helps people find great recipes based on their needs.

User query:
"{user_query}"

Here are some recipes you can consider:
===RECIPE START===
{top_contexts[0]}
===RECIPE END===
===RECIPE START===
{top_contexts[1]}
===RECIPE END===
===RECIPE START===
{top_contexts[2]}
===RECIPE END===

Now write a helpful recommendation for one of these recipes, explaining why it
meets the user's request.
At the end, include the recipe title, ingredients, and steps.
[/INST]"""

In [49]:
# load chatbot model
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

NameError: name 'AutoTokenizer' is not defined

In [ ]:
# test
user_query = "I'd like something with mushrooms - what do you recommend?"
top_contexts = get_top_recipes(embedding_model, user_query)
prompt = create_prompt(user_query, top_contexts)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
output = model.generate(
    **inputs,
    max_new_tokens=500,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    repetition_penalty=1.1
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are a friendly cooking assistant that helps people find great recipes based on their needs.

User query:
"I'd like something with mushrooms - what do you recommend?"

Here are some recipes you can consider:
===RECIPE START===
Title: Homemade Cream of Mushroom Soup
Description: Make and share this Homemade Cream of Mushroom Soup recipe from Food.com.        
Tags: Low Protein, Winter, Savory, < 30 Mins, Stove Top, Easy
Ingredients: fresh mushrooms, onions, garlic clove, butter, flour, chicken broth, evaporated milk, salt, pepper, nutmeg
Steps: Cut the mushrooms into slices., Melt butter in large frying pan.  Add in onions, garlic, and mushrooms.  Cook until onions are soft., Blend in 2 T. flour and stir., Add in the chicken broth and heat until slightly thickened while stirring frequently., Stir cream with additional 1 T. flour and seasonings.  Add in cream to soup.  Heat to thicken while stirring frequently., Serve and enjoy!
===RECIPE END===
===RECIPE START===
Title: Classic Bake

In [48]:
print(tokenizer.decode(output[0], skip_special_tokens=True)).strip()

NameError: name 'tokenizer' is not defined

### Looking into gradio

In [ ]:
!pip install gradio
